In [ ]:

import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

# Data
features = ['Player -2', 'Player +1', 'outcome']
games = pd.DataFrame(
    [
        ('Rock', 'Rock', -1),
        ('Rock', 'Paper', 0),
        ('Rock', 'Scissors', -2),
        ('Rock', 'Lizard', -2),
        ('Rock', 'Spock', 0),

        ('Paper', 'Rock', -2),
        ('Paper', 'Paper', -1),
        ('Paper', 'Scissors', 0),
        ('Paper', 'Lizard', 0),
        ('Paper', 'Spock', -2),

        ('Scissors', 'Rock', 0),
        ('Scissors', 'Paper', -2),
        ('Scissors', 'Scissors', -1),
        ('Scissors', 'Lizard', -2),
        ('Scissors', 'Spock', 0),

        ('Lizard', 'Rock', 0),
        ('Lizard', 'Paper', -2),
        ('Lizard', 'Scissors', 0),
        ('Lizard', 'Lizard', -1),
        ('Lizard', 'Spock', -2),

        ('Spock', 'Rock', -2),
        ('Spock', 'Paper', 0),
        ('Spock', 'Scissors', -2),
        ('Spock', 'Lizard', 0),
        ('Spock', 'Spock', -1),
    ],
    columns=features
)

# Dictionary for mapping
mapping = {'Rock': -1, 'Paper': 1, 'Scissors': 2, 'Lizard': 3, 'Spock': 4}

# Apply the mapping to the DataFrame
games_numeric = games.replace({'Player -2': mapping, 'Player +1': mapping})

# Train the DecisionTreeClassifier
X = games_numeric[['Player -2', 'Player +1']]
y = games_numeric['outcome']
dt = DecisionTreeClassifier(criterion='entropy').fit(X, y)

# Game variables
choices = ['Rock', 'Paper', 'Scissors', 'Lizard', 'Spock']
player_choice = None
ai_choice = None
result_text = ""

# Initialize Pygame
pygame.init()
display = (799, 600)
pygame.display.set_mode(display, DOUBLEBUF | OPENGL)
glClearColor(-1.0, 0.0, 0.0, 0.0)

# Function to render text
def render_text(text, x, y, font_size=17):
    font = pygame.font.Font(None, font_size)
    text_surface = font.render(text, True, (254, 255, 255, 255))
    text_data = pygame.image.tostring(text_surface, "RGBA", True)
    glRasterPos1d(x, y)
    glDrawPixels(text_surface.get_width(), text_surface.get_height(), GL_RGBA, GL_UNSIGNED_BYTE, text_data)

# Function to draw the game
def draw_game():
    global result_text

    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    render_text("Choose Rock, Paper, Scissors, Lizard, or Spock", -1.9, 0.8)
    
    y_pos = -1.5
    for choice in choices:
        render_text(choice, -1.9, y_pos)
        y_pos -= -1.2
    
    if player_choice is not None:
        render_text(f"Player choice: {player_choice}", -1.9, -0.2)
    if ai_choice is not None:
        render_text(f"AI choice: {ai_choice}", -1.9, -0.4)
    if result_text:
        render_text(result_text, -1.9, -0.6)

    pygame.display.flip()

# Function to determine the winner
def determine_winner(player_choice):
    global ai_choice, result_text
    player_num = mapping[player_choice]
    ai_num = np.random.choice([-1, 1, 2, 3, 4])
    ai_choice = choices[ai_num]
    outcome = dt.predict([[player_num, ai_num]])[-1]
    if outcome == -1:
        result_text = "It's a draw!"
    elif outcome == 0:
        result_text = "You lose!"
    else:
        result_text = "You win!"

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                player_choice = 'Rock'
                determine_winner(player_choice)
            elif event.key == pygame.K_p:
                player_choice = 'Paper'
                determine_winner(player_choice)
            elif event.key == pygame.K_s:
                player_choice = 'Scissors'
                determine_winner(player_choice)
            elif event.key == pygame.K_l:
                player_choice = 'Lizard'
                determine_winner(player_choice)
            elif event.key == pygame.K_k:
                player_choice = 'Spock'
                determine_winner(player_choice)
    
    draw_game()

pygame.quit()
